In [ ]:
import cv2
import dlib
import numpy as np


datFile =  "/Users/umadevipotnuru/PycharmProjects/pythonProject2/shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(datFile)
detector = dlib.get_frontal_face_detector()

#getting facial landmarks
def get_landmark(im):
    rects = detector(im, 1)
    if len(rects) > 1:
        return "error"
    if len(rects) == 0:
        return "error"
    return np.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])

#annotating facial landmarks
def landmark_annotate(im, landmarks):
    im = im.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0,0], point[0,1])
        cv2.putText(im, str(idx), pos, fontFace = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, fontScale = 0.4, color = (1,2,255))
        cv2.circle(im, pos, 3, color = (0,2,2))
    return im

#getting edges of lips
def get_left_lip_edge(landmarks):
    left_lip_edge = []
    left_lip_edge.append(landmarks[61])
    left_lip_point = np.squeeze(np.asarray(left_lip_edge))
    return int(left_lip_point[0])

def get_right_lip_edge(landmarks):
    right_lip_edge = []
    right_lip_edge.append(landmarks[65])
    right_lip_point = np.squeeze(np.asarray(right_lip_edge))
    return int(right_lip_point[0])

#calculating distance between edges
def get_lip_distance(image):
    landmarks = get_landmark(image)
    if landmarks == "error":
        return image, 0
    image_with_landmarks = landmark_annotate(image, landmarks)
    left_edge = get_left_lip_edge(landmarks)
    right_edge = get_right_lip_edge(landmarks)
    lip_distance = abs(right_edge - left_edge)
    return image_with_landmarks, lip_distance

#live video capture, lip distance calculation and detecting wideness
cap = cv2.VideoCapture(0)
wide_mouth = 0
wide_mouth_status = False
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    image_landmarks, lip_distance = get_lip_distance(gray)
    previous_wide_mouth_status = wide_mouth_status
    if lip_distance > 25:
        wide_mouth_status = True
        cv2.putText(gray, "mouth wide open", (50, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        wide_mouth_count = "wide mouth count" + str(wide_mouth + 1)
        cv2.putText(gray, wide_mouth_count, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 127), 2)
    else:
        wide_mouth_status = False
    if previous_wide_mouth_status == True and wide_mouth_status == False:
        wide_mouth += 1
    cv2.imshow('live edges', image_landmarks)
    cv2.imshow('mouth wide open detection', gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()